# Where would you place your office in Amsterdam?

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/Orsi/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geograp

In [3]:
amsterdam_df = pd.read_csv('amsterdam.csv')

In [4]:
amsterdam_df.head()

subject region_name regio_type region_code  ninhabitants  \
0              Amsterdam   Amsterdam   Gemeente      GM0363        833624   
1  Burgwallen-Oude Zijde   Amsterdam       Wijk    WK036300          4280   
2            Kop Zeedijk   Amsterdam      Buurt  BU03630000          1020   
3         Oude Kerk e.o.   Amsterdam      Buurt  BU03630001           670   
4        Burgwallen Oost   Amsterdam      Buurt  BU03630002          1610   

     nmen  nwomen  nage_0_to_15  nage_15_to_25  nage_25_to_45  nage_45_to_65  \
0  411608  422016        127224         108370         293154         204741   
1    2340    1935           255            675           2030            980   
2     570     445            50            140            520            260   
3     365     300            30            130            320            145   
4     880     730           120            250            755            340   

   nage_65_older  nunmarried  nmarried  ndivorced  nwidowed  \
0         100135      531112    203974      73962     24576   
1            355        3280       660        290        65   
2             60         765       175         70        15   
3             55         540        70         55        15   
4            155        1230       255        115        25   

   nimmigrant_western  nimmigrant_nonwestern  nimmigrant_marokko  \
0              140901                 289823               75065   
1                1275                    640                  25   
2                 320                    145                   5   
3                 205                    105                   5   
4                 495                    275                  10   

   nimmigrant_antiles_aruba  nimmigrant_surinam  nimmigrant_turkey  \
0                     12227               66119              42622   
1                        40                  80                 40   
2                        10                  20                 10   
3                        10                  15                 10   
4                        15                  35                 10   

   nimmigrant_other_non_western  nhouseholds  nhh_single_person  \
0                         93790       456462             251828   
1                           455         3070               2160   
2                           105          740                520   
3                            70          500                370   
4                           210         1120                770   

   nhh_no_children  nhh_with_children  ave_househ_size  populatio_density  \
0            92709             111925              1.8             5042.0   
1              655                265              1.4            12255.0   
2              170                 55              1.4            16818.0   
3               95                 45              1.3             7615.0   
4              260                105              1.4            25867.0   

   area_total  area_land  area_water  urbanisation_grade  address_density  \
0       21949      16533        5416                 1.0           6011.0   
1          40         35           6                 1.0           7619.0   
2           8          6           2                 1.0           6649.0   
3           9          9           0                 1.0           7361.0   
4           7          6           1                 1.0           7544.0   

                                             geojson       lon        lat  
0  {"type": "FeatureCollection","crs": { "type": ...  4.879568  52.381000  
1  {"type": "FeatureCollection","crs": { "type": ...  4.896943  52.372566  
2  {"type": "FeatureCollection","crs": { "type": ...  4.900280  52.375937  
3  {"type": "FeatureCollection","crs": { "type": ...  4.896156  52.373793  
4  {"type": "FeatureCollection","crs": { "type": ...  4.898757  52.372815

In [6]:
amsterdam_df.shape

(579, 37)

In [5]:
print('The dataframe has {} regions and {} neighborhoods and {} regio types.'.format(
        len(amsterdam_df['region_name'].unique()),
        len(amsterdam_df['subject'].unique()),
        len(amsterdam_df['regio_type'].unique())
    )
)

The dataframe has 1 regions and 574 neighborhoods and 3 regio types.


### The latitude and longitude values are available from the dataset

We create a map of Amsterdam

In [8]:
latitude = 52.379189
longitude = 4.899431

# create map of Amsterdam using latitude and longitude values
map_amsterdam = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, region, neighborhood in zip(amsterdam_df['lat'], amsterdam_df['lon'], amsterdam_df['region_name'], amsterdam_df['subject']):
    label = '{}, {}'.format(neighborhood, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

#### Define Foursquare Credentials and Version

In [44]:
CLIENT_ID = 'PNJVHZA5NZMNJ5RA2INBMZ3VQLYWKQRPN3UXYJGAJ54PFK4U' # your Foursquare ID
CLIENT_SECRET = 'VVCS0GHRQTNSN5PPK3NGHH2X2XF1BUUYH314C1V4YUD0WBTH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PNJVHZA5NZMNJ5RA2INBMZ3VQLYWKQRPN3UXYJGAJ54PFK4U
CLIENT_SECRET:VVCS0GHRQTNSN5PPK3NGHH2X2XF1BUUYH314C1V4YUD0WBTH


**First neighborhood**

In [45]:
amsterdam_df.loc[0, 'subject']

'Amsterdam'

In [46]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    amsterdam_df.loc[0, 'lat'], 
    amsterdam_df.loc[0, 'lon'], 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PNJVHZA5NZMNJ5RA2INBMZ3VQLYWKQRPN3UXYJGAJ54PFK4U&client_secret=VVCS0GHRQTNSN5PPK3NGHH2X2XF1BUUYH314C1V4YUD0WBTH&v=20180605&ll=52.38099994141871,4.87956802018358&radius=500&limit=100'

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8ef21f0f59680026922a68'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Frederik Hendrikbuurt',
  'headerFullLocation': 'Frederik Hendrikbuurt, Amsterdam',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 71,
  'suggestedBounds': {'ne': {'lat': 52.38549994591871,
    'lng': 4.886926382909526},
   'sw': {'lat': 52.376499936918705, 'lng': 4.872209657457634}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '553ac522498eb1b18ede4019',
       'name': 'Checkpoint Charlie',
       'location': {'address': 'Nassaukade 48',
        'lat': 52.38216642986923,
        'lng': 4.878820957859672,
        'labeledLatLngs': [{'label': 'display

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/Orsi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


name             categories        lat       lng
0  Checkpoint Charlie                    Bar  52.382166  4.878821
1               Broer                    Bar  52.379797  4.876983
2               Moods            Coffee Shop  52.379525  4.880572
3       Terang Boelan  Indonesian Restaurant  52.379685  4.882610
4          Svaha Yoga            Yoga Studio  52.381002  4.882808

In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

71 venues were returned by Foursquare.


#### Let's explore the neighborhoods in Amsterdam

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        if 'response' in requests.get(url).json():
            if 'groups' in requests.get(url).json()['response']:
                # make the GET request
                results = requests.get(url).json()['response']['groups'][0]['items']

                # return only relevant information for each nearby venue
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [36]:
amsterdam_df['subject'][0:5]

0                Amsterdam
1    Burgwallen-Oude Zijde
2              Kop Zeedijk
3           Oude Kerk e.o.
4          Burgwallen Oost
Name: subject, dtype: object

In [49]:
# type your answer here

amsterdam_venues = getNearbyVenues(names=amsterdam_df['subject'],
                                   latitudes=amsterdam_df['lat'],
                                   longitudes=amsterdam_df['lon']
                                  )

Amsterdam
Burgwallen-Oude Zijde
Kop Zeedijk
Oude Kerk e.o.
Burgwallen Oost
Nes e.o.
BG-terrein e.o.
Burgwallen-Nieuwe Zijde
Stationsplein e.o.
Hemelrijk
Nieuwendijk Noord
Spuistraat Noord
Nieuwe Kerk e.o.
Spuistraat Zuid
Begijnhofbuurt
Kalverdriehoek
Grachtengordel-West
Langestraat e.o.
Leliegracht e.o.
Felix Meritisbuurt
Leidsegracht Noord
Grachtengordel-Zuid
Spiegelbuurt
Gouden Bocht
Van Loonbuurt
Amstelveldbuurt
Rembrandtpleinbuurt
Reguliersbuurt
Leidsegracht Zuid
Nieuwmarkt/Lastage
Oosterdokseiland
Scheepvaarthuisbuurt
Rapenburg
Lastage
Nieuwmarkt
Uilenburg
Valkenburg
Zuiderkerkbuurt
Waterloopleinbuurt
Haarlemmerbuurt
Westerdokseiland
Haarlemmerbuurt Oost
Haarlemmerbuurt West
Westelijke eilanden
Planciusbuurt Noord
Planciusbuurt Zuid
Jordaan
Driehoekbuurt
Bloemgrachtbuurt
Marnixbuurt Noord
Zaagpoortbuurt
Marnixbuurt Midden
Elandsgrachtbuurt
Passeerdersgrachtbuurt
Groenmarktkadebuurt
Marnixbuurt Zuid
Anjeliersbuurt Noord
Anjeliersbuurt Zuid
De Weteringschans
Leidsebuurt Noordwest
Le

In [50]:
print(amsterdam_venues.shape)
amsterdam_venues.head()

(15460, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Amsterdam                 52.381                4.879568   
1    Amsterdam                 52.381                4.879568   
2    Amsterdam                 52.381                4.879568   
3    Amsterdam                 52.381                4.879568   
4    Amsterdam                 52.381                4.879568   

                Venue  Venue Latitude  Venue Longitude         Venue Category  
0  Checkpoint Charlie       52.382166         4.878821                    Bar  
1               Broer       52.379797         4.876983                    Bar  
2               Moods       52.379525         4.880572            Coffee Shop  
3       Terang Boelan       52.379685         4.882610  Indonesian Restaurant  
4          Svaha Yoga       52.381002         4.882808            Yoga Studio

In [51]:
amsterdam_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                    
Aalsmeerwegbuurt Oost                                      44   
Aalsmeerwegbuurt West                                      34   
Afrikahaven                                                 1   
Alexanderplein e.o.                                        56   
Alfa-driehoek                                               5   
Amstelveldbuurt                                            76   
Amsterdam                                                  71   
Anjeliersbuurt Noord                                      100   
Anjeliersbuurt Zuid                                       100   
Apollobuurt                                                20   
Architectenbuurt                                            6   
BG-terrein e.o.                                           100   
Balboaplein e.o.                                           41   
Banpleinbuurt                                              53   
Bedrijvencentrum Westerkwartier                            19   
Bedrijvengebied Cruquiusweg                                 5   
Bedrijvengebied Veelaan                                    35   
Bedrijvengebied Zeeburgerkade                              27   
Bedrijventerrein Landlust                                  22   
Bedrijventerrein Schinkel                                  12   
Bedrijventerrein Sloterdijk                                 5   
Bedrijventerrein Sloterdijk I                               7   
Beethovenbuurt                                             25   
Begijnhofbuurt                                            100   
Bellamybuurt Noord                                        100   
Bellamybuurt Zuid                                         100   
Bertelmanpleinbuurt                                        37   
Bloemgrachtbuurt                                           89   
Borgerbuurt                                               100   
Borneo                                                      4   
Bosleeuw                                                   42   
Bretten Oost                                                5   
Bretten West                                                5   
Buiteneiland                                                1   
Burgemeester Tellegenbuurt Oost                            33   
Burgemeester Tellegenbuurt West                            40   
Burgwallen Oost                                           100   
Burgwallen-Nieuwe Zijde                                   100   
Burgwallen-Oude Zijde                                     100   
Buyskade e.o.                                              37   
Centrale Markt                                             18   
Centrumeiland                                              12   
Coenhaven/Mercuriushaven                                    4   
Columbusplein e.o.                                         46   
Concertgebouwbuurt                                        100   
Cornelis Schuytbuurt                                       47   
Cornelis Troostbuurt                                       66   
Cremerbuurt Oost                                           54   
Cremerbuurt West                                           40   
Czaar Peterbuurt                                           45   
Da Costabuurt                                              91   
Da Costabuurt Noord                                        91   
Da Costabuurt Zuid                                        100   
Dapperbuurt                                                43   
Dapperbuurt Noord                                          53   
Dapperbuurt Zuid                                           39   
De Eenhoorn                                                23   
De Heining                                                  4   
De Kolenkit                                                28   
De Wetbuurt                                                12   
De Weterin

In [52]:
print('There are {} uniques categories.'.format(len(amsterdam_venues['Venue Category'].unique())))

There are 332 uniques categories.


### Let's analyze each neighborhood

In [56]:
# one hot encoding
amsterdam_onehot = pd.get_dummies(amsterdam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amsterdam_onehot['Neighborhood'] = amsterdam_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [amsterdam_onehot.columns[-1]] + list(amsterdam_onehot.columns[:-1])
amsterdsam_onehot = amsterdam_onehot[fixed_columns]

amsterdam_onehot.head()

Accessories Store  Afghan Restaurant  African Restaurant  \
0                  0                  0                   0   
1                  0                  0                   0   
2                  0                  0                   0   
3                  0                  0                   0   
4                  0                  0                   0   

   American Restaurant  Antique Shop  Aquarium  Arcade  \
0                    0             0         0       0   
1                    0             0         0       0   
2                    0             0         0       0   
3                    0             0         0       0   
4                    0             0         0       0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                       0            0           0                    0   
1                       0            0           0                    0   
2                       0            0           0                    0   
3                       0            0           0                    0   
4                       0            0           0                    0   

   Arts & Entertainment  Asian Restaurant  Athletics & Sports  \
0                     0                 0                   0   
1                     0                 0                   0   
2                     0                 0                   0   
3                     0                 0                   0   
4                     0                 0                   0   

   Australian Restaurant  Austrian Restaurant  Auto Workshop  Automotive Shop  \
0                      0                    0              0                0   
1                      0                    0              0                0   
2                      0                    0              0                0   
3                      0                    0              0                0   
4                      0                    0              0                0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bar  Basketball Court  Beach  \
0          0           0           0       0    1                 0      0   
1          0           0           0       0    1                 0      0   
2          0           0           0       0    0                 0      0   
3          0           0           0       0    0                 0      0   
4          0           0           0       0    0                 0      0   

   Bed & Breakfast  Beer Bar  Beer Garden  Beer Store  Belgian Restaurant  \
0                0         0            0           0                   0   
1                0         0            0           0                   0   
2                0         0            0           0                   0   
3                0         0            0           0                   0   
4                0         0            0           0                   0   

   Bike Rental / Bike Share  Bike Shop  Bistro  Board Shop  Boarding House  \
0                         0          0       0           0               0   
1                         0          0       0           0               0   
2                         0          0       0           0               0   
3                         0          0       0           0               0   
4                         0          0       0           0               0   

   Boat or Ferry  Bookstore  Botanical Garden  Boutique  Bowling Alley  \
0              0          0                 0         0              0   
1              0          0                 0         0              0   
2              0          0                 0         0              0   
3              0          0                 0         0              0   
4              0          0                 0         0              0   

   Brasserie  Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0          0            

In [57]:
amsterdam_onehot.shape

(15460, 332)

#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [59]:
amsterdam_grouped = amsterdam_onehot.groupby('Neighborhood').mean().reset_index()
amsterdam_grouped.head()

Neighborhood  Accessories Store  Afghan Restaurant  \
0  Aalsmeerwegbuurt Oost                0.0                0.0   
1  Aalsmeerwegbuurt West                0.0                0.0   
2            Afrikahaven                0.0                0.0   
3    Alexanderplein e.o.                0.0                0.0   
4          Alfa-driehoek                0.0                0.0   

   African Restaurant  American Restaurant  Antique Shop  Aquarium  Arcade  \
0            0.000000                  0.0           0.0  0.000000     0.0   
1            0.000000                  0.0           0.0  0.000000     0.0   
2            0.000000                  0.0           0.0  0.000000     0.0   
3            0.017857                  0.0           0.0  0.017857     0.0   
4            0.000000                  0.0           0.0  0.000000     0.0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                0.022727          0.0         0.0                  0.0   
1                0.029412          0.0         0.0                  0.0   
2                0.000000          0.0         0.0                  0.0   
3                0.000000          0.0         0.0                  0.0   
4                0.000000          0.0         0.0                  0.0   

   Arts & Entertainment  Asian Restaurant  Athletics & Sports  \
0                   0.0               0.0            0.022727   
1                   0.0               0.0            0.000000   
2                   0.0               0.0            0.000000   
3                   0.0               0.0            0.000000   
4                   0.0               0.0            0.000000   

   Australian Restaurant  Austrian Restaurant  Auto Workshop  Automotive Shop  \
0               0.022727                  0.0            0.0              0.0   
1               0.029412                  0.0            0.0              0.0   
2               0.000000                  0.0            0.0              0.0   
3               0.000000                  0.0            0.0              0.0   
4               0.000000                  0.0            0.0              0.0   

   BBQ Joint  Baby Store  Bagel Shop    Bakery       Bar  Basketball Court  \
0        0.0         0.0    0.022727  0.022727  0.045455               0.0   
1        0.0         0.0    0.029412  0.029412  0.058824               0.0   
2        0.0         0.0    0.000000  0.000000  1.000000               0.0   
3        0.0         0.0    0.000000  0.017857  0.035714               0.0   
4        0.0         0.0    0.000000  0.000000  0.000000               0.0   

   Beach  Bed & Breakfast  Beer Bar  Beer Garden  Beer Store  \
0    0.0              0.0  0.000000     0.000000         0.0   
1    0.0              0.0  0.000000     0.000000         0.0   
2    0.0              0.0  0.000000     0.000000         0.0   
3    0.0              0.0  0.017857     0.017857         0.0   
4    0.0              0.0  0.000000     0.000000         0.0   

   Belgian Restaurant  Bike Rental / Bike Share  Bike Shop  Bistro  \
0                 0.0                       0.0   0.022727     0.0   
1                 0.0                       0.0   0.029412     0.0   
2                 0.0                       0.0   0.000000     0.0   
3                 0.0                       0.0   0.000000     0.0   
4                 0.0                       0.0   0.000000     0.0   

   Board Shop  Boarding House  Boat or Ferry  Bookstore  Botanical Garden  \
0         0.0             0.0            0.0        0.0               0.0   
1         0.0             0.0            0.0        0.0               0.0   
2         0.0             0.0            0.0        0.0               0.0   
3         0.0             0.0            0.0        0.0               0.0   
4         0.0             0.0            0.0        0.0               0.0   

   Boutique  Bowling Alley  Brasserie  Brazilian Restaurant  Breakfast Spot  \


In [60]:
amsterdam_grouped.shape

(320, 332)

In [61]:
num_top_venues = 5

for hood in amsterdam_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = amsterdam_grouped[amsterdam_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aalsmeerwegbuurt Oost----
          venue  freq
0   Supermarket  0.07
1  Tram Station  0.05
2   Pizza Place  0.05
3    Restaurant  0.05
4           Bar  0.05


----Aalsmeerwegbuurt West----
         venue  freq
0  Supermarket  0.12
1         Café  0.06
2          Bar  0.06
3   Bagel Shop  0.03
4     Pharmacy  0.03


----Afrikahaven----
                           venue  freq
0                            Bar   1.0
1                    Opera House   0.0
2  Paper / Office Supplies Store   0.0
3                         Palace   0.0
4                Paintball Field   0.0


----Alexanderplein e.o.----
         venue  freq
0  Zoo Exhibit  0.23
1  Supermarket  0.05
2         Café  0.05
3   Restaurant  0.04
4          Bar  0.04


----Alfa-driehoek----
                           venue  freq
0                    Music Venue   0.4
1  Paper / Office Supplies Store   0.2
2                      Nightclub   0.2
3                       Ski Area   0.2
4                   Optical Shop   0.0


----Amst

Let's put the most common venues in a pandas data frame

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [98]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = amsterdam_grouped['Neighborhood']

for ind in np.arange(amsterdam_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amsterdam_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue          2nd Most Common Venue  \
0  Aalsmeerwegbuurt Oost           Supermarket                            Bar   
1  Aalsmeerwegbuurt West           Supermarket                           Café   
2            Afrikahaven                   Bar                    Zoo Exhibit   
3    Alexanderplein e.o.           Zoo Exhibit                    Supermarket   
4          Alfa-driehoek           Music Venue  Paper / Office Supplies Store   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0    Italian Restaurant           Pizza Place            Restaurant   
1                   Bar                 Hotel                Bakery   
2           Fish Market               Exhibit    Falafel Restaurant   
3                  Café            Restaurant               Theater   
4             Nightclub              Ski Area                 Field   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0          Tram Station      Sushi Restaurant             Bike Shop   
1             Bike Shop        Breakfast Spot    Miscellaneous Shop   
2                  Farm        Farmers Market  Fast Food Restaurant   
3                   Bar                 Hotel                   Zoo   
4           Event Space               Exhibit    Falafel Restaurant   

  9th Most Common Venue     10th Most Common Venue  
0               Brewery  Middle Eastern Restaurant  
1          Burger Joint                   Bus Stop  
2                 Field          Fish & Chips Shop  
3                Lounge              Tanning Salon  
4                  Farm             Farmers Market

### Clustering the neighborhoods of Amsterdam into four clusters

In [99]:
# set number of clusters
kclusters = 5

amsterdam_grouped_clustering = amsterdam_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(amsterdam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 0, 4, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 0, 0, 3, 0, 3,
       0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 2, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3,
       0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0,
       3, 3, 3, 0, 3, 1, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 3, 3,
       0, 3, 3, 0, 3, 3, 0, 0, 0, 3, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 1, 1, 0, 0, 0, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3,
       0, 0, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 3,
       0, 0, 3, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 3, 3, 3,
       3, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 0, 0, 0, 0, 0,

In [104]:
kmeans.labels_.shape

(320,)

In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [110]:
neighborhoods_venues_sorted.head()

Cluster Labels           Neighborhood 1st Most Common Venue  \
0               3  Aalsmeerwegbuurt Oost           Supermarket   
1               0  Aalsmeerwegbuurt West           Supermarket   
2               4            Afrikahaven                   Bar   
3               3    Alexanderplein e.o.           Zoo Exhibit   
4               3          Alfa-driehoek           Music Venue   

           2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                            Bar    Italian Restaurant           Pizza Place   
1                           Café                   Bar                 Hotel   
2                    Zoo Exhibit           Fish Market               Exhibit   
3                    Supermarket                  Café            Restaurant   
4  Paper / Office Supplies Store             Nightclub              Ski Area   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant          Tram Station      Sushi Restaurant   
1                Bakery             Bike Shop        Breakfast Spot   
2    Falafel Restaurant                  Farm        Farmers Market   
3               Theater                   Bar                 Hotel   
4                 Field           Event Space               Exhibit   

  8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
0             Bike Shop               Brewery  Middle Eastern Restaurant  
1    Miscellaneous Shop          Burger Joint                   Bus Stop  
2  Fast Food Restaurant                 Field          Fish & Chips Shop  
3                   Zoo                Lounge              Tanning Salon  
4    Falafel Restaurant                  Farm             Farmers Market

In [126]:
print(neighborhoods_venues_sorted.shape)
print(amsterdam_df.shape)

(320, 12)
(579, 37)


In [128]:
amsterdam_df.head()

subject region_name regio_type region_code  ninhabitants  \
0              Amsterdam   Amsterdam   Gemeente      GM0363        833624   
1  Burgwallen-Oude Zijde   Amsterdam       Wijk    WK036300          4280   
2            Kop Zeedijk   Amsterdam      Buurt  BU03630000          1020   
3         Oude Kerk e.o.   Amsterdam      Buurt  BU03630001           670   
4        Burgwallen Oost   Amsterdam      Buurt  BU03630002          1610   

     nmen  nwomen  nage_0_to_15  nage_15_to_25  nage_25_to_45  nage_45_to_65  \
0  411608  422016        127224         108370         293154         204741   
1    2340    1935           255            675           2030            980   
2     570     445            50            140            520            260   
3     365     300            30            130            320            145   
4     880     730           120            250            755            340   

   nage_65_older  nunmarried  nmarried  ndivorced  nwidowed  \
0         100135      531112    203974      73962     24576   
1            355        3280       660        290        65   
2             60         765       175         70        15   
3             55         540        70         55        15   
4            155        1230       255        115        25   

   nimmigrant_western  nimmigrant_nonwestern  nimmigrant_marokko  \
0              140901                 289823               75065   
1                1275                    640                  25   
2                 320                    145                   5   
3                 205                    105                   5   
4                 495                    275                  10   

   nimmigrant_antiles_aruba  nimmigrant_surinam  nimmigrant_turkey  \
0                     12227               66119              42622   
1                        40                  80                 40   
2                        10                  20                 10   
3                        10                  15                 10   
4                        15                  35                 10   

   nimmigrant_other_non_western  nhouseholds  nhh_single_person  \
0                         93790       456462             251828   
1                           455         3070               2160   
2                           105          740                520   
3                            70          500                370   
4                           210         1120                770   

   nhh_no_children  nhh_with_children  ave_househ_size  populatio_density  \
0            92709             111925              1.8             5042.0   
1              655                265              1.4            12255.0   
2              170                 55              1.4            16818.0   
3               95                 45              1.3             7615.0   
4              260                105              1.4            25867.0   

   area_total  area_land  area_water  urbanisation_grade  address_density  \
0       21949      16533        5416                 1.0           6011.0   
1          40         35           6                 1.0           7619.0   
2           8          6           2                 1.0           6649.0   
3           9          9           0                 1.0           7361.0   
4           7          6           1                 1.0           7544.0   

                                             geojson       lon        lat  
0  {"type": "FeatureCollection","crs": { "type": ...  4.879568  52.381000  
1  {"type": "FeatureCollection","crs": { "type": ...  4.896943  52.372566  
2  {"type": "FeatureCollection","crs": { "type": ...  4.900280  52.375937  
3  {"type": "FeatureCollection","crs": { "type": ...  4.896156  52.373793  
4  {"type": "FeatureCollection","crs": { "type": ...  4.898757  52.372815

In [129]:
amsterdam_merged = amsterdam_df

In [130]:
amsterdam_merged = neighborhoods_venues_sorted.set_index('Neighborhood').join(amsterdam_merged.set_index('subject'))

In [131]:
amsterdam_merged.head() # check the last columns!

Cluster Labels 1st Most Common Venue  \
Aalsmeerwegbuurt Oost               3           Supermarket   
Aalsmeerwegbuurt West               0           Supermarket   
Afrikahaven                         4                   Bar   
Alexanderplein e.o.                 3           Zoo Exhibit   
Alfa-driehoek                       3           Music Venue   

                               2nd Most Common Venue 3rd Most Common Venue  \
Aalsmeerwegbuurt Oost                            Bar    Italian Restaurant   
Aalsmeerwegbuurt West                           Café                   Bar   
Afrikahaven                              Zoo Exhibit           Fish Market   
Alexanderplein e.o.                      Supermarket                  Café   
Alfa-driehoek          Paper / Office Supplies Store             Nightclub   

                      4th Most Common Venue 5th Most Common Venue  \
Aalsmeerwegbuurt Oost           Pizza Place            Restaurant   
Aalsmeerwegbuurt West                 Hotel                Bakery   
Afrikahaven                         Exhibit    Falafel Restaurant   
Alexanderplein e.o.              Restaurant               Theater   
Alfa-driehoek                      Ski Area                 Field   

                      6th Most Common Venue 7th Most Common Venue  \
Aalsmeerwegbuurt Oost          Tram Station      Sushi Restaurant   
Aalsmeerwegbuurt West             Bike Shop        Breakfast Spot   
Afrikahaven                            Farm        Farmers Market   
Alexanderplein e.o.                     Bar                 Hotel   
Alfa-driehoek                   Event Space               Exhibit   

                      8th Most Common Venue 9th Most Common Venue  \
Aalsmeerwegbuurt Oost             Bike Shop               Brewery   
Aalsmeerwegbuurt West    Miscellaneous Shop          Burger Joint   
Afrikahaven            Fast Food Restaurant                 Field   
Alexanderplein e.o.                     Zoo                Lounge   
Alfa-driehoek            Falafel Restaurant                  Farm   

                          10th Most Common Venue region_name regio_type  \
Aalsmeerwegbuurt Oost  Middle Eastern Restaurant   Amsterdam      Buurt   
Aalsmeerwegbuurt West                   Bus Stop   Amsterdam      Buurt   
Afrikahaven                    Fish & Chips Shop   Amsterdam      Buurt   
Alexanderplein e.o.                Tanning Salon   Amsterdam      Buurt   
Alfa-driehoek                     Farmers Market   Amsterdam      Buurt   

                      region_code  ninhabitants  nmen  nwomen  nage_0_to_15  \
Aalsmeerwegbuurt Oost  BU03634403          2225  1035    1185           280   
Aalsmeerwegbuurt West  BU03634402          2490  1125    1365           265   
Afrikahaven            BU03631006             5     5       0             0   
Alexanderplein e.o.    BU03630804           395   205     185            55   
Alfa-driehoek          BU03631001            10     5       5             5   

                       nage_15_to_25  nage_25_to_45  nage_45_to_65  \
Aalsmeerwegbuurt Oost            225           1020            515   
Aalsmeerwegbuurt West            270           1320            450   
Afrikahaven                        5              0              5   
Alexanderplein e.o.               35            180            100   
Alfa-driehoek                      0             10              0   

                       nage_65_older  nunmarried  nmarried  ndivorced  \
Aalsmeerwegbuurt Oost            190        1590       425        175   
Aalsmeerwegbuurt West            190        1840       430        185   
Afrikahaven                        0           5         5          5   
Alexanderplein e.o.               35         295        75         25   
Alfa-driehoek                      0          10         5          0   

                       nwidowed  nimmigrant_western  nimmigrant_nonwestern  \
Aalsmeerwegbuurt Oost        40                 455               

In [132]:
amsterdam_merged.shape

(321, 47)

In [138]:
amsterdam_merged = amsterdam_merged.reset_index()

In [139]:
amsterdam_merged.head()

index  Cluster Labels 1st Most Common Venue  \
0  Aalsmeerwegbuurt Oost               3           Supermarket   
1  Aalsmeerwegbuurt West               0           Supermarket   
2            Afrikahaven               4                   Bar   
3    Alexanderplein e.o.               3           Zoo Exhibit   
4          Alfa-driehoek               3           Music Venue   

           2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                            Bar    Italian Restaurant           Pizza Place   
1                           Café                   Bar                 Hotel   
2                    Zoo Exhibit           Fish Market               Exhibit   
3                    Supermarket                  Café            Restaurant   
4  Paper / Office Supplies Store             Nightclub              Ski Area   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant          Tram Station      Sushi Restaurant   
1                Bakery             Bike Shop        Breakfast Spot   
2    Falafel Restaurant                  Farm        Farmers Market   
3               Theater                   Bar                 Hotel   
4                 Field           Event Space               Exhibit   

  8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  \
0             Bike Shop               Brewery  Middle Eastern Restaurant   
1    Miscellaneous Shop          Burger Joint                   Bus Stop   
2  Fast Food Restaurant                 Field          Fish & Chips Shop   
3                   Zoo                Lounge              Tanning Salon   
4    Falafel Restaurant                  Farm             Farmers Market   

  region_name regio_type region_code  ninhabitants  nmen  nwomen  \
0   Amsterdam      Buurt  BU03634403          2225  1035    1185   
1   Amsterdam      Buurt  BU03634402          2490  1125    1365   
2   Amsterdam      Buurt  BU03631006             5     5       0   
3   Amsterdam      Buurt  BU03630804           395   205     185   
4   Amsterdam      Buurt  BU03631001            10     5       5   

   nage_0_to_15  nage_15_to_25  nage_25_to_45  nage_45_to_65  nage_65_older  \
0           280            225           1020            515            190   
1           265            270           1320            450            190   
2             0              5              0              5              0   
3            55             35            180            100             35   
4             5              0             10              0              0   

   nunmarried  nmarried  ndivorced  nwidowed  nimmigrant_western  \
0        1590       425        175        40                 455   
1        1840       430        185        45                 535   
2           5         5          5         0                   0   
3         295        75         25        10                 100   
4          10         5          0         0                   5   

   nimmigrant_nonwestern  nimmigrant_marokko  nimmigrant_antiles_aruba  \
0                    350                  65                        25   
1                    335                  50                        20   
2                      0                   0                         0   
3                     40                   0                         5   
4                     10                   0                         0   

   nimmigrant_surinam  nimmigrant_turkey  nimmigrant_other_non_western  \
0                  70                 25                           165   
1                  60                 15                           190   
2                   0                  0                             0   
3                  10                  5                            25   
4                   0                  0                            10   

   nhouseholds  nhh_single_person  nhh_no_children  nhh_with_children  \
0

In [140]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amsterdam_merged['lat'], amsterdam_merged['lon'], amsterdam_merged['index'], amsterdam_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

In [143]:
amsterdam_merged.shape

(321, 48)

### Cluster 1

In [149]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 0]

index  Cluster Labels  \
1                     Aalsmeerwegbuurt West               0   
5                           Amstelveldbuurt               0   
6                                 Amsterdam               0   
7                      Anjeliersbuurt Noord               0   
8                       Anjeliersbuurt Zuid               0   
9                               Apollobuurt               0   
10                         Architectenbuurt               0   
11                          BG-terrein e.o.               0   
13                            Banpleinbuurt               0   
16                  Bedrijvengebied Veelaan               0   
17            Bedrijvengebied Zeeburgerkade               0   
18                Bedrijventerrein Landlust               0   
20              Bedrijventerrein Sloterdijk               0   
22                           Beethovenbuurt               0   
23                           Begijnhofbuurt               0   
24                       Bellamybuurt Noord               0   
25                        Bellamybuurt Zuid               0   
26                      Bertelmanpleinbuurt               0   
27                         Bloemgrachtbuurt               0   
28                              Borgerbuurt               0   
34          Burgemeester Tellegenbuurt Oost               0   
35          Burgemeester Tellegenbuurt West               0   
36                          Burgwallen Oost               0   
37                  Burgwallen-Nieuwe Zijde               0   
38                    Burgwallen-Oude Zijde               0   
44                       Concertgebouwbuurt               0   
45                     Cornelis Schuytbuurt               0   
46                     Cornelis Troostbuurt               0   
47                         Cremerbuurt Oost               0   
48                         Cremerbuurt West               0   
50                            Da Costabuurt               0   
51                      Da Costabuurt Noord               0   
52                       Da Costabuurt Zuid               0   
53                              Dapperbuurt               0   
54                        Dapperbuurt Noord               0   
55                         Dapperbuurt Zuid               0   
56                              De Eenhoorn               0   
57                               De Heining               0   
59                              De Wetbuurt               0   
60                        De Weteringschans               0   
61                     De Wittenbuurt Noord               0   
62                      De Wittenbuurt Zuid               0   
63                             Den Texbuurt               0   
64                             Diamantbuurt               0   
66                                Don Bosco               0   
67                            Driehoekbuurt               0   
68                            Duivelseiland               0   
69                                  Ecowijk               0   
70                        Elandsgrachtbuurt               0   
75                    Fannius Scholtenbuurt               0   
76                       Felix Meritisbuurt               0   
80                          Frans Halsbuurt               0   
81                    Frederik Hendrikbuurt               0   
82              Frederik Hendrikbuurt Noord               0   
83           Frederik Hendrikbuurt Zuidoost               0   
84           Frederik Hendrikbuurt Zuidwest               0   
85                      Frederikspleinbuurt               0   
86                          Gerard Doubuurt               0   
87                              Geuzenbuurt               0   
88                           Geuzenhofbuurt               0   
89                           Gibraltarbuurt               0   
90                             Gouden Bocht               0   
91                      Grachtengordel-West               0   
92    

In [150]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 1]

index  Cluster Labels 1st Most Common Venue  \
116            IJburg Oost               1                 Beach   
167  Middeneiland Zuidoost               1                 Beach   
168  Middeneiland Zuidwest               1                 Beach   
255           Strandeiland               1                 Beach   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
116        Ice Cream Shop       Harbor / Marina     Convenience Store   
167            Campground            Skate Park           Zoo Exhibit   
168            Campground            Skate Park           Zoo Exhibit   
255            Campground            Skate Park           Zoo Exhibit   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
116            Campground              Bus Stop            Skate Park   
167     Fish & Chips Shop    Falafel Restaurant                  Farm   
168     Fish & Chips Shop    Falafel Restaurant                  Farm   
255     Fish & Chips Shop    Falafel Restaurant                  Farm   

    8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
116     Fish & Chips Shop                  Farm         Farmers Market   
167        Farmers Market  Fast Food Restaurant                  Field   
168        Farmers Market  Fast Food Restaurant                  Field   
255        Farmers Market  Fast Food Restaurant                  Field   

    region_name regio_type region_code  ninhabitants  nmen  nwomen  \
116   Amsterdam       Wijk    WK036350             0     0       0   
167   Amsterdam      Buurt  BU03635001             0     0       0   
168   Amsterdam      Buurt  BU03635003             0     0       0   
255   Amsterdam      Buurt  BU03635000             0     0       0   

     nage_0_to_15  nage_15_to_25  nage_25_to_45  nage_45_to_65  nage_65_older  \
116             0              0              0              0              0   
167             0              0              0              0              0   
168             0              0              0              0              0   
255             0              0              0              0              0   

     nunmarried  nmarried  ndivorced  nwidowed  nimmigrant_western  \
116           0         0          0         0                   0   
167           0         0          0         0                   0   
168           0         0          0         0                   0   
255           0         0          0         0                   0   

     nimmigrant_nonwestern  nimmigrant_marokko  nimmigrant_antiles_aruba  \
116                      0                   0                         0   
167                      0                   0                         0   
168                      0                   0                         0   
255                      0                   0                         0   

     nimmigrant_surinam  nimmigrant_turkey  nimmigrant_other_non_western  \
116                   0                  0                             0   
167                   0                  0                             0   
168                   0                  0                             0   
255                   0                  0                             0   

     nhouseholds  nhh_single_person  nhh_no_children  nhh_with_children  \
116            0                  0                0                  0   
167            0                  0                0                  0   
168            0                  0                0                  0   
255            0                  0                0                  0   

     ave_househ_size  populatio_density  area_total  area_land  area_water  \
116              NaN                NaN          27         27           0   
167              NaN                NaN           0          0           0   
168              NaN                NaN           6          6           0   
255              Na

In [151]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 2]

index  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
33  Buiteneiland               2         Boat or Ferry           Zoo Exhibit   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
33           Flea Market               Exhibit    Falafel Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
33                  Farm        Farmers Market  Fast Food Restaurant   

   9th Most Common Venue 10th Most Common Venue region_name regio_type  \
33                 Field      Fish & Chips Shop   Amsterdam      Buurt   

   region_code  ninhabitants  nmen  nwomen  nage_0_to_15  nage_15_to_25  \
33  BU03635005             0     0       0             0              0   

    nage_25_to_45  nage_45_to_65  nage_65_older  nunmarried  nmarried  \
33              0              0              0           0         0   

    ndivorced  nwidowed  nimmigrant_western  nimmigrant_nonwestern  \
33          0         0                   0                      0   

    nimmigrant_marokko  nimmigrant_antiles_aruba  nimmigrant_surinam  \
33                   0                         0                   0   

    nimmigrant_turkey  nimmigrant_other_non_western  nhouseholds  \
33                  0                             0            0   

    nhh_single_person  nhh_no_children  nhh_with_children  ave_househ_size  \
33                  0                0                  0              NaN   

    populatio_density  area_total  area_land  area_water  urbanisation_grade  \
33                NaN           4          4           0                 NaN   

    address_density                                            geojson  \
33              NaN  {"type": "FeatureCollection","crs": { "type": ...   

        lon        lat  
33  5.00809  52.367151

In [152]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 3]

index  Cluster Labels  \
0               Aalsmeerwegbuurt Oost               3   
3                 Alexanderplein e.o.               3   
4                       Alfa-driehoek               3   
12                   Balboaplein e.o.               3   
14    Bedrijvencentrum Westerkwartier               3   
15        Bedrijvengebied Cruquiusweg               3   
19          Bedrijventerrein Schinkel               3   
21      Bedrijventerrein Sloterdijk I               3   
29                             Borneo               3   
30                           Bosleeuw               3   
31                       Bretten Oost               3   
32                       Bretten West               3   
39                      Buyskade e.o.               3   
40                     Centrale Markt               3   
41                      Centrumeiland               3   
42           Coenhaven/Mercuriushaven               3   
43                 Columbusplein e.o.               3   
49                   Czaar Peterbuurt               3   
58                        De Kolenkit               3   
65                   Diepenbrockbuurt               3   
71                 Entrepot-Noordwest               3   
72                        Erasmuspark               3   
73              Erasmusparkbuurt Oost               3   
74              Erasmusparkbuurt West               3   
77                          Flevopark               3   
78                        Frankendael               3   
79                        Frankendael               3   
98                  Haveneiland Noord               3   
101                  Haveneiland Oost               3   
108                         Het Funen               3   
111               Hoofddorppleinbuurt               3   
112                     Hoofdweg e.o.               3   
113                        Houthavens               3   
115                   Houthavens West               3   
118                       IJburg Zuid               3   
123               Indische Buurt Oost               3   
125                   Jan Maijenbuurt               3   
131                       KNSM-eiland               3   
132                          Kadijken               3   
134                        Kattenburg               3   
135                      Kazernebuurt               3   
137               Kolenkitbuurt Noord               3   
138                Kolenkitbuurt Zuid               3   
142                 Laan van Spartaan               3   
144                    Landlust Noord               3   
145                     Landlust Zuid               3   
162                       Markthallen               3   
166                      Mercatorpark               3   
175            Nieuwe Diep/Diemerpark               3   
181  Noordoostkwadrant Indische buurt               3   
184            Olympisch Stadion e.o.               3   
186      Oostelijke Eilanden/Kadijken               3   
188                        Oostenburg               3   
196              Orteliusbuurt Midden               3   
197               Orteliusbuurt Noord               3   
198                Orteliusbuurt Zuid               3   
201           Overbraker Binnenpolder               3   
204              Paramariboplein e.o.               3   
207          Pieter van der Doesbuurt               3   
208               Planciusbuurt Noord               3   
213                   RI Oost terrein               3   
217                   Rieteiland Oost               3   
219                         Rijnbuurt               3   
220                  Rijnbuurt Midden               3   
223            Robert Scottbuurt Oost               3   
224            Robert Scottbuurt West               3   
228                      Scheldebuurt               3   
234                Schinkelbuurt Zuid               3   
235                        Sloterdijk               3   
236                     Sloterdijk II     

In [153]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 4]

index  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
2  Afrikahaven               4                   Bar           Zoo Exhibit   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
2           Fish Market               Exhibit    Falafel Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
2                  Farm        Farmers Market  Fast Food Restaurant   

  9th Most Common Venue 10th Most Common Venue region_name regio_type  \
2                 Field      Fish & Chips Shop   Amsterdam      Buurt   

  region_code  ninhabitants  nmen  nwomen  nage_0_to_15  nage_15_to_25  \
2  BU03631006             5     5       0             0              5   

   nage_25_to_45  nage_45_to_65  nage_65_older  nunmarried  nmarried  \
2              0              5              0           5         5   

   ndivorced  nwidowed  nimmigrant_western  nimmigrant_nonwestern  \
2          5         0                   0                      0   

   nimmigrant_marokko  nimmigrant_antiles_aruba  nimmigrant_surinam  \
2                   0                         0                   0   

   nimmigrant_turkey  nimmigrant_other_non_western  nhouseholds  \
2                  0                             0            0   

   nhh_single_person  nhh_no_children  nhh_with_children  ave_househ_size  \
2                  5                0                  5              1.8   

   populatio_density  area_total  area_land  area_water  urbanisation_grade  \
2                NaN         536        420         116                 5.0   

   address_density                                            geojson  \
2             23.0  {"type": "FeatureCollection","crs": { "type": ...   

        lon        lat  
2  4.745073  52.412312